# Analizando la página objetivo

Este notebook tiene como objetivo analizar la página a la cual queremos hacer webscrapping:

https://www.isciii.es/QueHacemos/Servicios/VigilanciaSaludPublicaRENAVE/EnfermedadesTransmisibles/Paginas/InformesCOVID-19.aspx

Esta página contiene los datos actualizados por parte del gobierno de los casos de contagio por coronavirus. El obejtivo de este notebook es en primer lugar, realziar webscrapping para descargar los PDFs que redirige. Seguidamente, para recolectar los datos del pdf realizaremos pdf scraping. Todos los pasos se automatizan también en scripts en el proyecto.

In [1]:
# libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
from bs4.element import Tag

In [2]:
# Descargamos la página objetivo
page = requests.get(
    'https://www.isciii.es/QueHacemos/Servicios/VigilanciaSaludPublicaRENAVE/'
    'EnfermedadesTransmisibles/Paginas/InformesCOVID-19.aspx'
)
if page.status_code == 200:
    print('Página descargada con éxito')

Página descargada con éxito


In [3]:
# creamos la sopa a traves de la página
soup = BeautifulSoup(page.content)

Las páginas objetivo se encuentran dentro de un a href.

In [4]:
# # buscamos bajo el id donde están los links de interés
# id_links = soup.find(id='ctl00_PlaceHolderMain_ctl02__ControlWrapper_RichHtmlField')

# # para cada hijo buscamos a y guardamos su enlace
# links_pdfs = []
# for child in id_links.children:
#     a_tag = child.find('a')
#     if isinstance(a_tag, Tag):
#         links_pdfs.append(a_tag.get('href'))

# print(links_pdfs[0])

In [5]:
# buscamos bajo el id donde están los links de interés
id_links = soup.find(id='ctl00_PlaceHolderMain_ctl02__ControlWrapper_RichHtmlField')

# para cada hijo buscamos a y guardamos su enlace
pdfs = []
for child in id_links.children:
    pdf = {}
    a_tag = child.find('a')
    if isinstance(a_tag, Tag):
        pdf['link'] = a_tag.get('href')
        pdf['info'] = a_tag.text
        pdfs.append(pdf)

from pprint import pprint
pprint(pdfs[0])

{'info': 'Informe nº\xa017. Situación de COVID-19 en España a 27 de marzo de '
         '2020',
 'link': '/QueHacemos/Servicios/VigilanciaSaludPublicaRENAVE/EnfermedadesTransmisibles/Documents/INFORMES/Informes%20COVID-19/Informe%20nº%2017.%20Situación%20de%20COVID-19%20en%20España%20a%2027%20marzo%20de%202020.pdf'}


In [6]:
# generamos cada enlace
baselink = 'https://www.isciii.es'
for pdf in pdfs:
    link = pdf['link']
    pdf.update({'link': baselink + link})
pprint(pdfs[0])

{'info': 'Informe nº\xa017. Situación de COVID-19 en España a 27 de marzo de '
         '2020',
 'link': 'https://www.isciii.es/QueHacemos/Servicios/VigilanciaSaludPublicaRENAVE/EnfermedadesTransmisibles/Documents/INFORMES/Informes%20COVID-19/Informe%20nº%2017.%20Situación%20de%20COVID-19%20en%20España%20a%2027%20marzo%20de%202020.pdf'}


In [7]:
## generamos cada enlace
#baselink = 'https://www.isciii.es'
#for i in range(0, len(links_pdfs)):
    #links_pdfs[i] = baselink + links_pdfs[i]
#print(links_pdfs[0])

In [8]:
import requests
name = pdfs[0]['info']
url = pdfs[0]['link']
file = requests.get(url, allow_redirects=True)
open('../data/' + name + '.pdf', 'wb').write(file.content)

1845825

Ahora que ya hemos descargado un pdf vamos a hacer pdf scrapping para extraer los datos.

In [9]:
import tabula
contenido_pdf = tabula.read_pdf('../data/' + name + '.pdf', pages='all', output_format='json')

Got stderr: mar. 28, 2020 12:09:33 P. M. org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFORMACIÓN: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
mar. 28, 2020 12:09:33 P. M. org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFORMACIÓN: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
mar. 28, 2020 12:09:33 P. M. org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFORMACIÓN: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
mar. 28, 2020 12:09:34 P. M. org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFORMACIÓN: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
mar. 28, 2020 12:09:34 P. M. org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFORMACIÓN: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
mar. 28,

In [14]:
# let's look for the interesting table
comunidades = [
    'Andalucía',
    'Aragón',
    'Asturias',
    'Baleares',
    'Canarias',
    'Cantabria',
    'Castilla La Mancha',
    'Castilla y León',
    'Cataluña',
    'Comunitat Valenciana',
    'Extremadura',
    'Galicia',
    'Madrid',
    'Murcia',
    'Navarra',
    'País Vasco',
    'La Rioja',
    'Ceuta',
    'Melilla'
]
datos_interes = []
for top_level_element in contenido_pdf:
    for level_1_element in top_level_element['data']:
        comunidad = {}
        for i in range(0, len(level_1_element)):
            if level_1_element[i]['text'] in comunidades:
                comunidad['comunidad'] = level_1_element[i]['text']
                comunidad['casos'] = level_1_element[i + 1]['text']
                comunidad['casos_notificados'] = level_1_element[i + 2]['text']
                datos_interes.append(comunidad)
                
datos_interes

[{'comunidad': 'Andalucía', 'casos': '3793', 'casos_notificados': '3145'},
 {'comunidad': 'Aragón', 'casos': '1338', 'casos_notificados': '482'},
 {'comunidad': 'Asturias', 'casos': '900', 'casos_notificados': '841'},
 {'comunidad': 'Baleares', 'casos': '755', 'casos_notificados': '325'},
 {'comunidad': 'Canarias', 'casos': '878', 'casos_notificados': '278'},
 {'comunidad': 'Cantabria', 'casos': '810', 'casos_notificados': '737'},
 {'comunidad': 'Castilla La Mancha',
  'casos': '3934',
  'casos_notificados': '0'},
 {'comunidad': 'Castilla y León',
  'casos': '4132',
  'casos_notificados': '3266'},
 {'comunidad': 'Cataluña', 'casos': '12940', 'casos_notificados': '1928'},
 {'comunidad': 'Comunitat Valenciana',
  'casos': '3532',
  'casos_notificados': '2453'},
 {'comunidad': 'Extremadura', 'casos': '1231', 'casos_notificados': '1199'},
 {'comunidad': 'Galicia', 'casos': '2322', 'casos_notificados': '132'},
 {'comunidad': 'Madrid', 'casos': '19243', 'casos_notificados': '8926'},
 {'comun